## 使用 `czsc` 进行选股
---


In [1]:
#!export http_proxy=http://nas.vvmm.ink:7890,https_proxy=http://nas.vvmm.ink:7890,all_proxy=socks5://nas.vvmm.ink:7891;
!pip install ta-lib mplfinance
!pip install git+git://github.com/sohunjug/czsc.git -U
!pip install tqdm
!pip install numba

  Cloning git://github.com/sohunjug/czsc.git to /tmp/pip-req-build-ftv2p84y
  Created wheel for czsc: filename=czsc-0.5.9-py3-none-any.whl size=33336 sha256=fa59991c5c5b2384a6c819c4cd54393571ccf7983c1be7c54e0b6d20c52ee9b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-20_nqok6/wheels/23/22/55/0530d7668e82d869fefb8602f04bc2c7bfbca7c51b66150f83
Successfully built czsc
  Attempting uninstall: czsc
    Found existing installation: czsc 0.5.8
    Uninstalling czsc-0.5.8:
      Successfully uninstalled czsc-0.5.8


In [ ]:
import sys
sys.path.insert(0, ".")
sys.path.insert(0, "..")

import czsc
print(czsc.__version__)
print(dir(czsc.data))
from czsc.data.ts import *
from czsc.data.bs import *
from datetime import datetime
from typing import List
import traceback
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from czsc.analyze import KlineAnalyze
import tushare as ts
# 如果是第一次使用需要设置 token
ts.set_token("565ee6d69fb85cb0bc7fdfc8dea4b8ce1f18366f30e8d23a253bb9cf")

In [3]:
# 导入 Tushare 数据（推荐使用）
from czsc.data.ts import *


In [4]:
# 导入聚宽数据
from czsc.data.jq import *

# 如果是第一次使用需要设置 token
set_token("18688989717", "hwqdir5.")

In [5]:
# 导入数据



ModuleNotFoundError: No module named 'czsc.data.bs'

### 缠论日线笔中枢三买选股
---

条件描述： 最近五笔走势，前三笔构成中枢，第四笔离开中枢，第五笔向下不回中枢



In [5]:
def selector1(symbols: List):
    """输入股票列表，输入符合买点定义的股票"""
    res = []
    for symbol in tqdm(symbols, desc="缠论日线笔中枢三买选股"):
        try:
            kline = get_kline(symbol=symbol, end_date=datetime.now(), freq="D", count=1000)
            ka = KlineAnalyze(kline, ma_params=(5, 34, 120, 233), bi_mode="new")
            points = ka.bi_list[-6:]
            
            if len(points) == 6 and points[-1]['fx_mark'] == "d":
                zs_g = min([x['bi'] for x in points[:4] if x['fx_mark'] == 'g'])
                zs_d = max([x['bi'] for x in points[:4] if x['fx_mark'] == 'd'])
                
                if points[-1]['bi'] > zs_g > zs_d:
                    res.append(symbol)
                    
        except:
            print("{} 分析失败".format(symbol))
            traceback.print_exc()
    return res

### 日线笔中枢选股
---

中枢如果能当下确认，基本逻辑如下：
1. ma 233<close<ma 60 ---得 codes_list1
2. codes_list1中选择 符合日线笔中枢形态的，得codes_list2
    * 2.1 笔中枢形态：方向下: 日线中枢完成，第四笔底分型
3. codes_list2 中中枢第一笔到第四笔得日期跨度 date1
4. date1时间跨度中，codes_list2 ma233 cross的个数，并标注 codes_list3

In [6]:
def cross_number(x1, x2):
    """输入两个序列，计算 x1 下穿 x2 的次数"""
    x = np.array(x1) < np.array(x2)
    num = 0
    for i in range(len(x)-1):
        b1, b2 = x[i], x[i+1]
        if b2 and b1 != b2:
            num += 1
    return num

x1 = [1, 1, 3, 4, 5, 12, 9, 8]
x2 = [2, 2, 1, 5, 8, 9, 10, 3]
print(cross_number(x1, x2))

2


In [7]:
def selector(symbols: List):
    """输入股票列表，输入符合买点定义的股票"""
    res = []
    for symbol in tqdm(symbols, desc="缠论选股"):
        try:
            kline = get_kline(symbol=symbol, end_date=datetime.now(), freq="D", count=1000)
            ka = KlineAnalyze(kline, ma_params=(5, 34, 60, 250), bi_mode="new")

            if ka.ma[-1]['ma60'] >= ka.latest_price >= ka.ma[-1]['ma250']:
                # print("{} 满足条件1：ma60 > close > ma233".format(symbol))
                points = ka.bi_list[-7:]
                
                if len(points) == 7 and points[-1]['fx_mark'] == 'd':
                    zs_g = min([x['bi'] for x in points[2:6] if x['fx_mark'] == 'g'])
                    zs_d = max([x['bi'] for x in points[2:6] if x['fx_mark'] == 'd'])
                    
                    if zs_g > zs_d:
                        # print("{} 满足条件2：向下中枢完成".format(symbol))
                        date_span = [points[-5]['dt'], points[-1]['dt']]
                        low = [x['low'] for x in ka.kline_raw if date_span[1] >= x['dt'] >= date_span[0]]
                        ma_ = [x['ma250'] for x in ka.ma if date_span[1] >= x['dt'] >= date_span[0]]
                        num = cross_number(low, ma_)
                        res.append({"symbol": symbol, "cross_num": num})
        except:
            print("{} 分析失败".format(symbol))
            traceback.print_exc()
    return res

## 执行选股
---

In [11]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector1(symbols)

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)


选股结果： ['000708.XSHE', '000768.XSHE', '002008.XSHE', '002230.XSHE', '002415.XSHE', '002460.XSHE', '002508.XSHE', '300014.XSHE', '300015.XSHE', '600009.XSHG', '600025.XSHG', '600031.XSHG', '600036.XSHG', '600104.XSHG', '600309.XSHG', '600346.XSHG', '600406.XSHG', '600660.XSHG', '600703.XSHG', '600741.XSHG', '600809.XSHG', '600893.XSHG', '601021.XSHG', '601100.XSHG', '601919.XSHG', '603195.XSHG', '603799.XSHG', '603833.XSHG', '603899.XSHG', '603993.XSHG']


In [9]:
# 使用聚宽数据在创业板综指上选股
#symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector(symbols)

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)


选股结果： [{'symbol': '002371.XSHE', 'cross_num': 0}, {'symbol': '002773.XSHE', 'cross_num': 2}, {'symbol': '300413.XSHE', 'cross_num': 0}, {'symbol': '300433.XSHE', 'cross_num': 0}, {'symbol': '300676.XSHE', 'cross_num': 0}, {'symbol': '601901.XSHG', 'cross_num': 0}]


In [12]:
# 使用 tushare 数据在创业板综指上选股
symbols = get_index_stocks("000300.XSHG")
#print(symbols)
selected = selector1(symbols)

print("选股结果：", selected)
# df = pd.DataFrame(selected)
# df.to_excel("选股结果.xlsx", index=False)


选股结果： ['000333.XSHE', '000768.XSHE', '000858.XSHE', '002008.XSHE', '002304.XSHE', '002415.XSHE', '002460.XSHE', '002508.XSHE', '300015.XSHE', '600009.XSHG', '600019.XSHG', '600025.XSHG', '600031.XSHG', '600036.XSHG', '600104.XSHG', '600111.XSHG', '600309.XSHG', '600346.XSHG', '600362.XSHG', '600406.XSHG', '600660.XSHG', '600741.XSHG', '600809.XSHG', '600926.XSHG', '601021.XSHG', '601100.XSHG', '601111.XSHG', '601318.XSHG', '601600.XSHG', '601601.XSHG', '601899.XSHG', '601919.XSHG', '603195.XSHG', '603799.XSHG', '603833.XSHG', '603899.XSHG', '603993.XSHG']
